探究代码如何实现loupe中的归一化

In [ ]:
import torch
import torch.nn as nn

class Memc_LOUPE(nn.Module):
    def __init__(self, input_shape, slope, sample_slope, device, sparsity):
        super(Memc_LOUPE, self).__init__()
        self.input_shape = input_shape
        self.slope = slope 
        self.device = device
        self.add_weight_real = nn.Parameter(- torch.log(1. / torch.rand(self.input_shape, dtype=torch.float32) - 1.) / self.slope, requires_grad=True)
        self.add_weight_imag = nn.Parameter(- torch.log(1. / torch.rand(self.input_shape, dtype=torch.float32) - 1.) / self.slope, requires_grad=True)
        self.sample_slope = sample_slope
        self.sparsity = sparsity
        self.conv = nn.Conv2d(4, 2, 1, 1, 0)

    def calculate_Mask(self, kspace_mc, option):
        print('sparsity:',self.sparsity)
        logit_weights_real = 0 * kspace_mc[:,0, :, :] + self.add_weight_real
        logit_weights_imag = 0 * kspace_mc[:,1, :, :] + self.add_weight_imag
        prob_mask_tensor = torch.cat((logit_weights_real, logit_weights_imag), dim=1)
        prob_mask_tensor = self.conv(prob_mask_tensor)
        prob_mask_tensor = torch.sigmoid(self.slope * prob_mask_tensor)
        #分析变量对应关系
        
        xbar = torch.mean(prob_mask_tensor)
        r = self.sparsity / xbar
        beta = (1 - self.sparsity) / (1 - xbar)
        le = (torch.less_equal(r, 1)).to(dtype=torch.float32)
        prob_mask_tensor = le * prob_mask_tensor * r + (1 - le) * (1 - (1 - prob_mask_tensor) * beta)
        threshs = torch.rand(prob_mask_tensor.size(), dtype=torch.float32).to(device=self.device)
        thresh_tensor = 0 * prob_mask_tensor + threshs

        last_tensor_mask = torch.sigmoid(self.sample_slope * (prob_mask_tensor - thresh_tensor)) 
        return last_tensor_mask.to(device=self.device)

    def forward(self,mask):
        B,C,H,W=mask.shape
        assert H==256 and W==256
        dcmask = self.calculate_Mask(mask, option=True)#inital work
        return  dcmask

探究nn.Parameter生成的参数是否是同时优化，同一个梯度不同的初始值？还是不同的初始值？ 实际需要应该是按照每个点用不同的梯度进行更新，卷积？ 默认前面的已经学出来概率分布，看看归一化是怎么实现的

In [ ]:
import 